In [16]:
import wandb
import torch
import numpy as np
import diffusion_pde as dpde
from wandb.apis.public.runs import Runs
from wandb.apis.public.artifacts import Artifacts, RunArtifacts, ArtifactFiles
#from hydra import initialize, compose
from omegaconf import OmegaConf

In [2]:
wandb.login()
api = wandb.Api()

wandb: Currently logged in as: philiphohwy (philiphohwy-danmarks-tekniske-universitet-dtu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
#with initialize(config_path="../conf", version_base=None):
#    cfg = compose(
#        config_name="train", 
#        overrides=[
#            "dataset=heat_eq_logt_joint",
#            "model=unetv2"
#        ]
#    )

wandb_cfg = OmegaConf.load("../conf/train.yaml").wandb

In [4]:
filters = {}

runs = Runs(
    client=api.client,
    entity=wandb_cfg.entity,
    project=wandb_cfg.project,
    filters=filters
)

In [5]:
for run in runs:
    print(f"Run ID: {run.id}, Name: {run.name}")

Run ID: hodjisac, Name: heat-logt/forward/unet-v2
Run ID: 8nb62ytp, Name: heat-logt/forward/unet-v2
Run ID: 4labp6a8, Name: heat-logt/joint/unet-v2


In [11]:
run_idx = 0
run_cfg = OmegaConf.create(runs[run_idx].config)

artifacts = RunArtifacts(
    client=api.client,
    run=runs[run_idx]
    )

for artifact in artifacts:
    print(f"Artifact Name: {artifact.name}, Type: {artifact.type}")

Artifact Name: heat-logt-forward-unet-v2:v0, Type: model
Artifact Name: run-hodjisac-history:v0, Type: wandb-history


In [13]:
print(OmegaConf.to_yaml(run_cfg))

lr: 0.0001
model:
  name: unet-v2
  emb_ch: 256
  obs_ch: 0
  base_ch: 64
  dropout: 0
  ch_mults:
  - 1
  - 2
  - 2
  noise_ch: 64
  sigma_data: 0.5
  n_res_blocks: 2
wandb:
  dir: /home/s204790/dynamical-pde-diffusion/logs
  name: None
  entity: philiphohwy-danmarks-tekniske-universitet-dtu
  project: dynamical-pde-diffusion-final-final
epochs: 1000
dataset:
  net:
    in_ch: 1
    label_ch: 2
  data:
    pde: heat
    name: heat_logt
    datapath: data/heat_logt.hdf5
  loss: edm
  method: forward
  sampling:
    ch_a: 1
    zeta_a: 50
    zeta_u: 40
    zeta_pde: 0.4
    loss_func: diffusion_pde.sampling.heat_loss
    num_steps: 50
    batch_size: 32
    sample_shape:
    - 2
    - 64
    - 64
  training:
    shuffle: true
    batch_size: 64
    num_epochs: 1000
    physics_loss: false
    weight_decay: 0
    learning_rate: 0.0001
    physics_loss_coeff: 1
  start_at_t0: true
run_name: heat-logt/forward/unet-v2
weight_decay: 0
model_save_path: /home/s204790/dynamical-pde-diffusion/p

In [7]:
model_path = artifacts[0].download(root=f"../pretrained_models/{runs[run_idx].id}/")

wandb:   1 of 1 files downloaded.  


In [15]:
print(model_path)

../pretrained_models/hodjisac/


In [17]:
edm = dpde.utils.get_net_from_config(run_cfg)
edm.load_state_dict(torch.load(f"{model_path}/model.pth", weights_only=True))

<All keys matched successfully>